<a href="https://colab.research.google.com/github/thegreatgupta/CBIR/blob/master/CBIR_Colab_02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Content Based Image Recommendation

Content Based Image Recommendation or CBIR is a new way of Search Engine which enables a user to Search for similar images over the network with the help of images directly. 

**Clone the Github Repository to Google Colab along with the Dataset.**

In [1]:
!git clone https://github.com/thegreatgupta/CBIR.git

Cloning into 'CBIR'...
remote: Enumerating objects: 60163, done.
remote: Counting objects: 100% (60163/60163), done.
remote: Compressing objects: 100% (60059/60059), done.
remote: Total 60163 (delta 106), reused 60154 (delta 103), pack-reused 0
Receiving objects: 100% (60163/60163), 159.37 MiB | 31.70 MiB/s, done.
Resolving deltas: 100% (106/106), done.
Checking out files: 100% (60215/60215), done.


In [1]:
!ls

CBIR  sample_data


In [2]:
cd CBIR

/content/CBIR


In [3]:
!ls

CBIR_01.ipynb	 LICENSE	      model_trail.py	 weight.h5
dataset		 model		      model_training.py
json_model.json  model_evaluation.py  README.md


### Import Required Packages

In [4]:
import pandas as pd
import numpy as np
import cv2
import os
import keras
import matplotlib.pyplot as plt
from keras.applications.mobilenet import MobileNet, preprocess_input
from keras.applications.imagenet_utils import decode_predictions
from keras.layers import Dense,GlobalAveragePooling2D, Dropout
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array, save_img
from keras.models import Model, load_model
from keras.optimizers import Adam

Using TensorFlow backend.


#### Step - 1 : Data Preprocessing

In [0]:
#BASE_PATH = 'F:/GIT/CBIR/'
BASE_PATH = ''
annotation = pd.read_csv(BASE_PATH + 'dataset/annotation.txt', delimiter='\t') 

In [6]:
annotation.head()

,filename,color,micro_category,macro_category,macro_category(english),Unnamed: 5,Unnamed: 6,Unnamed: 7
0,35154736FEF.jpg,BRONZO,GONNA PELLE,ABBIGLIAMENTO PELLE,leather,NaN,NaN,NaN
1,35157444XDF.jpg,PLATINO,GONNA PELLE,ABBIGLIAMENTO PELLE,leather,NaN,NaN,NaN
2,35157749QFF.jpg,NOCCIOLA,GONNA PELLE,ABBIGLIAMENTO PELLE,leather,NaN,NaN,NaN
3,35165506NDF.jpg,ORO,GONNA PELLE,ABBIGLIAMENTO PELLE,leather,NaN,NaN,NaN
4,35167181CRF.jpg,ANTRACITE,GONNA PELLE,ABBIGLIAMENTO PELLE,leather,NaN,NaN,NaN


In [7]:
annotation = annotation[['filename', 'color', 'micro_category', 'macro_category', 'macro_category(english)']]
annotation.head()

,filename,color,micro_category,macro_category,macro_category(english)
0,35154736FEF.jpg,BRONZO,GONNA PELLE,ABBIGLIAMENTO PELLE,leather
1,35157444XDF.jpg,PLATINO,GONNA PELLE,ABBIGLIAMENTO PELLE,leather
2,35157749QFF.jpg,NOCCIOLA,GONNA PELLE,ABBIGLIAMENTO PELLE,leather
3,35165506NDF.jpg,ORO,GONNA PELLE,ABBIGLIAMENTO PELLE,leather
4,35167181CRF.jpg,ANTRACITE,GONNA PELLE,ABBIGLIAMENTO PELLE,leather


In [8]:
data_category_01 = annotation[['filename', 'macro_category(english)']]
data_category_01.head()

,filename,macro_category(english)
0,35154736FEF.jpg,leather
1,35157444XDF.jpg,leather
2,35157749QFF.jpg,leather
3,35165506NDF.jpg,leather
4,35167181CRF.jpg,leather


In [9]:
count = 0
for name, groups in data_category_01.groupby('macro_category(english)'):
    print(name)
    count += 1

print(count)

accessories
bags
coats
diaries
dresses
eye glasses
fitness
footwear
furnishings
furniture
gift ideas
jeans
jewellery
knitwear
leather
lighting
nursing
overalls
pants
pets
sea & swimming pool
shirts
skirts
sportswear
suitcases
suits
tables & kitchen
tailleur
tech gadget
textile
top wear
toys
watches/clocks
33


###### Below Code is commented.
It is used to convert the Dataset downloaded to required size and store in Local Directory.
It is done to convert and store the whole dataset at Github. This is done totally bacause of Computation.

```
i=0
count = 0
for filename in data_category_01.values:
    try:
        image_load_path = BASE_PATH + 'dataset/images/' + filename[0].split('.')[0] + '_resized.' + filename[0].split('.')[1]
        img = img_to_array(load_img(image_load_path, target_size=(128, 128, 3)))
        image_save_path = BASE_PATH + 'dataset/img_128_128/' + filename[0]
        save_img(image_save_path, img)
        i += 1
        if i % 1000 == 0:
            print(i)
    except:
        print('File Not Found ' + filename)
        count += 1
        
# This is formatted as code
```





In [0]:
image_array = np.ndarray(shape=(len(data_category_01), 64, 64, 3), dtype=float)

In [11]:
i=0
count = 0
for filename in data_category_01.values:
    try:
        #image_load_path = 'dataset/images/' + filename[0].split('.')[0] + '_resized.' + filename[0].split('.')[1]
        image_load_path = BASE_PATH + 'dataset/img_128_128/' + filename[0]
        #print(image_load_path)
        img = img_to_array(load_img(image_load_path, target_size=(64, 64, 3)))
        img = img / 255
        image_array[i] = img
        #np.append(image_array, img)
        i += 1
        if i % 1000 == 0:
            print(i)
    except:
        print('File Not Found ' + filename[0])
        count += 1
    
print('Count:' + str(count))

1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
Count:0


###### Split the Dataset into Train and Test Dataset and Save it into two different folders i.e. training_set and test_set

In [0]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(image_array, data_category_01, test_size=0.15)

In [0]:
image_array.resize(1)

In [14]:
image_array.shape

(1,)